# Ejemplo de implementación de una LSTM.

In [1]:
# ejemplo de una LSTM.
import numpy as np
class LSTMCell:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Weights for the input gate
        self.W_i = np.random.randn(hidden_size, input_size)
        self.U_i = np.random.randn(hidden_size, hidden_size)
        self.b_i = np.zeros((hidden_size, 1))

        # weights for the forget cell
        self.W_f = np.random.randn(hidden_size, input_size)
        self.U_f = np.random.randn(hidden_size, hidden_size)
        self.b_f = np.zeros((hidden_size, 1))

        # weights for the output cell
        self.W_o = np.random.randn(hidden_size, input_size)
        self.U_o = np.random.randn(hidden_size, hidden_size)
        self.b_o = np.zeros((hidden_size, 1))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def tanh(self, x):
        return np.tanh(x)

    def forward(self, x_t, h_prev, C_prev):
        # Calculate the input gate
        i_t = self.sigmoid(np.dot(self.W_i, x_t) + np.dot(self.U_i, h_prev) + self.b_i)

        # forget gate
        f_t = self.sigmoid(np.dot(self.W_f, x_t) + np.dot(self.U_f, h_prev) + self.b_f)

        # cell state candidate
        C_tilde_t = self.tanh(np.dot(self.W_o, x_t) + np.dot(self.U_o, h_prev) + self.b_o)

        # cell state
        C_t = f_t * C_prev + i_t * C_tilde_t

        # output gate
        o_t = self.sigmoid(np.dot(self.W_o, x_t) + np.dot(self.U_o, h_prev) + self.b_o)

        # hidden state
        h_t = o_t * self.tanh(C_t)

        return h_t, C_t




In [2]:
# test the algorithm.
input_size = 10
hidden_size = 20
output_size = 5

# create the lstm cell
lstm = LSTMCell(input_size, hidden_size, output_size)

# create some dummy data for one time step.
x_t = np.random.randn(input_size, 1)
h_prev = np.zeros((hidden_size, 1))
C_prev = np.zeros((hidden_size,1))

# forward pass through the LSTM cell
h_t, C_t = lstm.forward(x_t, h_prev, C_prev)

# print the output
print("h_t shape:", h_t.shape)
print("C_t shape:", C_t.shape)

# print the output
print("h_t:", h_t)
print("C_t:", C_t)

h_t shape: (20, 1)
C_t shape: (20, 1)
h_t: [[ 0.13136996]
 [ 0.01184139]
 [ 0.02738362]
 [ 0.73319839]
 [-0.00345482]
 [-0.03357348]
 [-0.02374232]
 [ 0.69504021]
 [ 0.25921575]
 [-0.11999735]
 [-0.09396889]
 [-0.12517915]
 [-0.00581224]
 [-0.05684264]
 [ 0.22688986]
 [-0.07297767]
 [ 0.05854894]
 [ 0.56128465]
 [-0.1099728 ]
 [ 0.08401951]]
C_t: [[ 0.13301207]
 [ 0.01187668]
 [ 0.04595907]
 [ 0.99379564]
 [-0.56651086]
 [-0.51034073]
 [-0.62139839]
 [ 0.87226591]
 [ 0.32185436]
 [-0.75541094]
 [-0.29152248]
 [-0.44051592]
 [-0.14382332]
 [-0.73617853]
 [ 0.28310232]
 [-0.72390191]
 [ 0.07455773]
 [ 0.68470556]
 [-0.30298592]
 [ 0.15268416]]


# Por que debería ser más estable un LSTM (con respecto al gradiente) que un RNN.

## Propagación hacia atrás.
Escribimos

$$J = \frac1T \sum_{t=1}^T J_t $$

Por linealidad basta que hallemos el gradiente para un $J_t$ arbitrario y luego se suma sobre todos los $t$s.

Por ahora pensemos solo en el forget cell. Los pesos son
$$W = [ W_f, U_f, b_f] $$



Vemos que $J_t = J_t(h_t, c_t, W) $
Aplicamos la regla de la cadena de la siguiente forma.

\begin{eqnarray}
\frac{\partial J_k}{\partial W} &=&
\frac{\partial J_k}{\partial h_k} \frac{\partial h_k}{\partial c_k}
\frac{\partial c_k}{\partial c_{k-1}} \cdots \frac{\partial c_2}{\partial c_1} \frac{\partial c_1}{\partial W} \\
&=& \frac{\partial J_k}{\partial h_k} \frac{\partial h_{k}}{\partial c_k} \prod_{t=k}^2 \frac{\partial c_t}{\partial c_{t-1}}   \frac{\partial c_1}{\partial W}
\end{eqnarray}

Nos enfocamos en $\partial c_t / \partial c_{t-1}$. Encontramos

\begin{eqnarray}
\frac{\partial c_t}{\partial c_{t-1}} &=& \frac{\partial}{\partial c_{t-1}} [ c_{t-1} \circ f_t + i_t  \circ \tilde{c}_t] \\
&=& \frac{\partial c_{t-1}}{\partial c_{t-1}} \circ f_t + c_{t-1} \circ \frac{\partial f_t}{\partial c_{t-1}} +
\frac{\partial i_t}{\partial c_{t-1}} \circ \tilde{c}_t + i_t \circ \frac{\partial \tilde{c}_t}{\partial c_{t-1}} \\
&=& I \circ f_t + c_{t-1}  \circ \frac{\partial f_t}{\partial c_{t-1}} + \frac{\partial i_t}{\partial c_{t-1}} \circ \tilde{c}_t + i_t \circ \frac{\partial \tilde{c}_t}{\partial c_{t-1}}
\end{eqnarray}



Acá $I$ es la matriz identidad en $\mathbb{R}^{h \times h}$, $I \circ f_tt$ es una matriz diagonal en $\mathbb{R}^{h \times h}$
Esta matriz diagonal tiene el mayor peso en el gradiente (como lo veremos a continuación) y si se quiere olvidar $f_t \approx 0$, si no se quiere olvidar $f_t \approx 1$. Veamos pues que los otros (3) términos  tienen poca contribución en el gradiente.

* $\frac{\partial f_t}{\partial c_{t-1}} $
Recuerden que

$$f_t = \sigma( W_f x_t + U_f h_{t-1} + b_f) = \sigma[W_f x_t + U_f \tanh(c_{t-1}) \circ o_{t-1} +  b_f] \  $$

Como vamos a tomar la derivada de esta expresión llamamos

$$z= W_f x_t + U_f \tanh(c_{t-1}) \circ o_{t-1} +  b_f $$

Al tomar la derivada vemos que

$$\frac{\partial f_t}{\partial c_{t-1}} = \sigma'(z) U_f \tanh'(c_{t-1})  (I \circ o_{t-1})    $$

La identidad viene de $\partial c_{t-1}/\partial c_{t-1}$.
Usamos los siguientes valores de referencia

$$0 \le \sigma'(z) \le 0.25  $$
$$0 \le \tanh'(z) \le 1 $$
$$0 \le o_{t-1} \le 1  $$

* $\frac{\partial i_t}{\partial c_{t-1}} $
Sabemos que

$$i_t = \sigma( W_i x_t + U_t h_{t-1} + b_i) =
 \sigma[ W_i x_t  + U_t \tanh(c_{t-1}) \circ o_{t-1} + b_i]$$
 De la misma forma que hicimos arriba

 $$z =   W_i x_t  + U_t \tanh(c_{t-1}) \circ o_{t-1} + b_i $$
 de forma que

 $$\frac{\partial i_t}{\partial c_{t-1}} = \sigma'(z) + U_t \tanh'(c_{t-1}) I  \circ o_{t-1}   $$

 El argumento basado en las desigualdades de arriba es el mismo. Este factor es pequeño relativamente y como está en una productoria, desvanece.

* $\frac{\partial \tilde{c}_t}{\partial c_{t-1}} $

De la clase anterior

$$\tilde{c}_t  = \tanh(W_c x_t + U_c h_{t-1} + b_c) = \tanh(W_c x_t + U_c \tanh(c_{t-1}) \circ o_{t-1} + b_c)   $$

De igual forma que hicimos arriba.

$$z = W_c x_t + U_c \tanh(c_{t-1}) \circ o_{t-1} + b_c $$

De forma que

$$\frac{\partial \tilde{c}_t}{\partial c_{t-1}} = \tanh'(z) U_c \tanh'(c_{t-1}) \circ o_{t-1}  $$

Usando los mismos argumentos de arriba $\tanh'(c_{t-1}) \in (0,1)$. La única forma que esta expresión sea grande es con los valores de $U_c$, si ese fuera el caso, hay un tipo de regularización que vimos en la clase anterior que es normalización de los pesos.

En **resumen**: El término dominante en la productoria de arriba (domina sobre el gradiente en el "long term") es $I \circ f_t$. Para valores de $f_t$ cercanos a 1 es muy estable.
Los otros términos son productos de factores en el intervalo $[0, 1/4]$, o $(0,1)$ y como estan multiplicados entre si, y dentro una productoria tienen a 0.

Los autores del LSTM dicen que esta red es estable para incluso mas de 1000 tiempos.

# Aplicaciones de las LSTM

* Generación de música.
    * [Paperspace Blog: Music Generation with LSTMs](https://blog.paperspace.com/music-generation-with-lstms/). Es un tutorial que TensorFlow y la library `pretty_midi` library.

    * [DataFlair: Automatic Music Generatiion Project Using Deep Learning](https://data-flair.training/blogs/automatic-music-generation-lstm-deep-learning/). Muestra paso a paso un proyecto que lee y procesa archivos MIDI (Musical Instrument Digital Interface) mediante la construcci'on de un modelo LSTM.

    * [Analytic Vidhya: Automatic Music Generation:](https://www.analyticsvidhya.com/blog/2020/01/how-to-perform-automatic-music-generation/)
    Este blog discute dos redes para la generación de música. WaveNet (??) y LSTM. Muestra ayudas de como implementar estos modelos y compara los resultados.

    * [The Kaggle: Music Generation with LSTM](https://www.kaggle.com/code/karnikakapoor/music-generation-lstm). Usa un Jupyter notebook interactivo y demuestra como funciona la LSTM. Incluye procesamiento de datos, entrenamiento de modlo y generación de nuevas secuencias musicales.

* Sentimental Analsis (analisis de "sentimientos"). Lo que hace es categorizar opiniones a traves de recursos computacionales. Las opiniones se extraen del texto indicando tendencias positivas, negativas o neutras. Veamos algunos ejemplos.

    * [Analytics Vidhya: Sentiment Analysis with LSTM](https://www.analyticsvidhya.com/blog/2022/01/sentiment-analysis-with-lstm/).  Este ttutrial muestra paso-a-paso como analizar los "sentimientos" usando LSTM en críticas (reviews) de IMDB datos. Estudia procesamiento, construcción del modelo con Keras, entrenamiento, evaluación y predicciones con datos nuevos.

    * [Kaggle: LSTM Sentiment Analysis with Keras](https://www.kaggle.com/code/ngyptr/lstm-sentiment-analysis-keras). Es interactivo con un cuaderno en Jupyter que demuestra el analisis de sentimientos usando LSTMs con Keras. Incluye, procesamiento, elaboración y entrenamiento del modelo, y evaluación en una base de datos IMDB.

    * [Embedded Robotics: Sentiment Analysis using LSTM, Tensorflow and Keras](https://www.embedded-robotics.com/sentiment-analysis-using-lstm/). Es un tutorial que construye LSTM bidireccional. Contiene lectura (carga) de datos, construcción del modelo, entrenamiento y evaluación.

    * [YouTube: Sentiement Analysis with LSTM | Deep Learning with Keras](https://www.youtube.com/watch?v=oWo9SNcyxlI).

* Predicción de acciones (stock prediction). Realmentte las LSTM no son muy buenas para esto. Por que las acciones no son estacionarias y tiene mucho ruido.

* Machine Translation: Esta era la columna vertebral de Google Translate. Esta herramienta es poderosa. El link [The medium.com blog](https://medium.com/@saikrishna4820/lstm-language-translation-18c076860b23) muestra un tutorial de traducción mediante el uso de LSTM.

* Predicción de trayectorias. Esto es importante para el software de carros autónomos. Hay muchos artículos en la internet de esto. Voy a citar uno. Alché and de La Fortelle, 2017.  

 F. Altché and A. de La Fortelle. An lstm network for highway trajectory prediction.
In 2017 IEEE 20th International Conference on Intelligent Transportation Systems
(ITSC), pages 353–359, 2017.

* Environmental Modeling: El artículo "An LSTM based aggregated model for air pollution forecasing" by Chang et al., 2029

Yue-Shan Chang, Hsin-Ta Chiao, Satheesh Abimannan, Yo-Ping Huang, Yi-Ting Tsai,
and Kuan-Ming Lin. An lstm-based aggregated model for air pollution forecasting.
Atmospheric Pollution Research, 11(8):1451–1463, 2020.

* Image Classification and Captioning: El arículo
[Automatic Image Captioning Base ond ResNet50 and LSTM with Soft Attention" por Chu, et al., 2020

Yan Chu, Xiao Yue, Lei Yu, Mikhailov Sergei, and Zhengkui Wang. Automatic image
captioning based on resnet50 and lstm with soft attention. Wireless Communications
and Mobile Computing, 2020:8909458, Oct 2020.

Esta referencia provee ideas interesanttes acerca de obtner etiquetas de imágnes a través de redes ResNet50 y LSTM.

* Voice Transciption: Puede convertir de texto a voz o vice-versa. [This Google Voice transcription blog](https://research.google/blog/the-neural-networks-behind-google-voice-transcription/) ofrece una discusión interesante acerca del useo de LSTM para la transcripción de audios.

* Google Voice Search: Convierte voz en texto para buscar.
El [blog](https://research.google/blog/google-voice-search-faster-and-more-accurate/) ofrece una discusión de como Google usa LSTM para esta aplicación.



# Ejemplo de LSTM para la predicción de la próxima palabra.
No estoy seguro que el ejemplo corra completamente en GoogleColab. Vamos a tratar.

El ejercicio es intersante por que va implicar:
* Bajar datos de Kaggle (la net)
* leer los datos (texto: SherlockHolmes.txt)
* Procesar texto.
* Crear los datos en grupos consecutivos de 3 palabras para predecir, la que sigue.
* Crear un modelo LSTM en `python` .
* Probar el modelo.

Este ejemplo se basa en
[este video](https://www.youtube.com/watch?v=Zn22qt7j2dM)

Pasos:

* Bajar los datos de Kaggle:
[Visit Kaggle](https://www.kaggle.com/datasets/adangonzalez/sherlock-holmes-txt) y baje los datos de allí.



In [3]:
# import libraries
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cd /root
!mkdir -p ~/.kaggle


Voy al sitio de Kaggle, log in, voy al avatar en la derecha superior y luego a settings, y hagoe click en New API token. Esto me baja el archivo kaggle.json con las credenciales de API.



In [8]:
import shutil
shutil.copyfile('/content/kaggle.json', '/root/.kaggle/kaggle.json')

'/root/.kaggle/kaggle.json'

In [9]:
!chmod 600 /root/.kaggle/kaggle.json

In [10]:
# leer el archivo
!kaggle datasets download -d adangonzalez/sherlock-holmes-txt

Dataset URL: https://www.kaggle.com/datasets/adangonzalez/sherlock-holmes-txt
License(s): unknown
  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 768MB/s]


In [11]:
# descomprimir el zip
!unzip sherlock-holmes-txt.zip

Archive:  sherlock-holmes-txt.zip
replace Sherlock.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [16]:
# abrir el archivo
file = open('/content/Sherlock.txt', 'r', encoding='utf-8')

In [17]:
file

<_io.TextIOWrapper name='/content/Sherlock.txt' mode='r' encoding='utf-8'>

In [19]:
# creamos lineas
lines = []
for i in file:
    lines.append(i)

lines[:20] #QC

['The Adventures of Sherlock Holmes\n',
 '\n',
 'by Arthur Conan Doyle\n',
 '\n',
 '\n',
 'Contents\n',
 '\n',
 '   I.     A Scandal in Bohemia\n',
 '   II.    The Red-Headed League\n',
 '   III.   A Case of Identity\n',
 '   IV.    The Boscombe Valley Mystery\n',
 '   V.     The Five Orange Pips\n',
 '   VI.    The Man with the Twisted Lip\n',
 '   VII.   The Adventure of the Blue Carbuncle\n',
 '   VIII.  The Adventure of the Speckled Band\n',
 '   IX.    The Adventure of the Engineer’s Thumb\n',
 '   X.     The Adventure of the Noble Bachelor\n',
 '   XI.    The Adventure of the Beryl Coronet\n',
 '   XII.   The Adventure of the Copper Beeches\n',
 '\n']

In [20]:
# juntamos las lineas
data = ""
for i in lines:
    data = ' '.join(lines)

data[:100]


'The Adventures of Sherlock Holmes\n \n by Arthur Conan Doyle\n \n \n Contents\n \n    I.     A Scandal in B'

In [21]:
# data processing
# remove backslashes and other things
data1 = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')
data1[:1000]

'The Adventures of Sherlock Holmes  by Arthur Conan Doyle   Contents     I.     A Scandal in Bohemia    II.    The Red-Headed League    III.   A Case of Identity    IV.    The Boscombe Valley Mystery    V.     The Five Orange Pips    VI.    The Man with the Twisted Lip    VII.   The Adventure of the Blue Carbuncle    VIII.  The Adventure of the Speckled Band    IX.    The Adventure of the Engineer’s Thumb    X.     The Adventure of the Noble Bachelor    XI.    The Adventure of the Beryl Coronet    XII.   The Adventure of the Copper Beeches     I. A SCANDAL IN BOHEMIA   I.  To Sherlock Holmes she is always _the_ woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the wor

In [22]:
# remover muliples espacios, separar palabras en lineas
data2 = data1.split()
data2[:10]

['The',
 'Adventures',
 'of',
 'Sherlock',
 'Holmes',
 'by',
 'Arthur',
 'Conan',
 'Doyle',
 'Contents']

In [23]:
# se vuelven a juntar las palabras pero ya sin espacios adicionales.
data = ' '.join(data2)
data[:1000]

'The Adventures of Sherlock Holmes by Arthur Conan Doyle Contents I. A Scandal in Bohemia II. The Red-Headed League III. A Case of Identity IV. The Boscombe Valley Mystery V. The Five Orange Pips VI. The Man with the Twisted Lip VII. The Adventure of the Blue Carbuncle VIII. The Adventure of the Speckled Band IX. The Adventure of the Engineer’s Thumb X. The Adventure of the Noble Bachelor XI. The Adventure of the Beryl Coronet XII. The Adventure of the Copper Beeches I. A SCANDAL IN BOHEMIA I. To Sherlock Holmes she is always _the_ woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position. He 

# Tokenization:
Las palabras se convierten a numeros. Queda una lista grande de numeros correspondientes a las palabras. La Tokenizacion es como la vectorizacion del lenguaje.

In [24]:
import itertools
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
word_index = tokenizer.word_index

# QC muestre la primeras 10 palabras con sus tokens
dict(itertools.islice(word_index.items(), 10))


{'the': 1,
 'and': 2,
 'to': 3,
 'of': 4,
 'i': 5,
 'a': 6,
 '”': 7,
 'in': 8,
 'that': 9,
 'it': 10}

In [25]:
sequential_data =  tokenizer.texts_to_sequences([data])[0]
sequential_data[:10]

[1, 1406, 4, 132, 34, 48, 698, 4604, 4605, 1844]

In [26]:
# la dimension del vector
len(sequential_data)

107995

In [27]:
# tamaño del vocabulario
vocab_size = len(word_index)
vocab_size

8642

In [29]:
# creae sequences of three words.
sequences = []
for i in range(3, len(sequential_data)):
    words = sequential_data[i-3:i+1]
    sequences.append(words)

len(sequences)

107992

In [30]:
# converir sequences a arreglos
sequences = np.array(sequences)
sequences[:10]

array([[   1, 1406,    4,  132],
       [1406,    4,  132,   34],
       [   4,  132,   34,   48],
       [ 132,   34,   48,  698],
       [  34,   48,  698, 4604],
       [  48,  698, 4604, 4605],
       [ 698, 4604, 4605, 1844],
       [4604, 4605, 1844,    5],
       [4605, 1844,    5,    6],
       [1844,    5,    6,  850]])

In [31]:
# define input output pairs.
X = []
y =[]
for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)

In [32]:
X[:10]

array([[   1, 1406,    4],
       [1406,    4,  132],
       [   4,  132,   34],
       [ 132,   34,   48],
       [  34,   48,  698],
       [  48,  698, 4604],
       [ 698, 4604, 4605],
       [4604, 4605, 1844],
       [4605, 1844,    5],
       [1844,    5,    6]])

In [33]:
y[:10]

array([ 132,   34,   48,  698, 4604, 4605, 1844,    5,    6,  850])

In [34]:
type(X)

numpy.ndarray

In [36]:
# convertir y a categorical (one-hot encoded)
y = to_categorical(y, num_classes=vocab_size + 1)
y[0] # QC

array([0., 0., 0., ..., 0., 0., 0.])

In [37]:
len(y[0])

8643

In [38]:
max(y[0])

np.float64(1.0)

In [39]:
np.argmax(y[0])

np.int64(132)

In [41]:
y[0][130:140]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])